In [3]:
!pip install pyqubo
!pip install rustworkx

!pip install docplex
!pip install qiskit
!pip install qiskit-optimization
!pip install shapely

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.7 MB/s eta 0:00:00ta 0:00:01


In [6]:
import csv
from shapely.wkt import loads

def parse_csv_file(file_path):
    # Initialize empty lists to store the extracted values
    latitudes = []
    longitudes = []
    names = []

    with open(file_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present

        for row in reader:
            # Extract the desired fields
            url = row[0]
            object_id = row[1]
            name = row[2]
            geom = loads(row[3])  # Parse the WKT geometry into a Shapely object
            line = row[4]
            notes = row[5]

            # Extract latitude and longitude from the Shapely point
            latitude = geom.y
            longitude = geom.x

            # Append the values to the respective lists
            latitudes.append(latitude)
            longitudes.append(longitude)
            names.append(name)

    return latitudes, longitudes, names

# Path to the CSV file
file_path = 'data/DOITT_SUBWAY_STATION_01_13SEPT2010.csv'

# Call the function to parse the data from the file
latitudes, longitudes, names = parse_csv_file(file_path)

coordinates = {}

for lat, lon, n in zip(latitudes, longitudes, names):
  coordinates[n] = [lat,lon]

# print(coordinates)


Calculating annual ridership for each station (f)

In [8]:
import csv
# Initialize an empty dictionary to store station sums
station_rider_sums = {}

# Open the CSV file for reading
with open('data/Annual Total-Table 1.csv', mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Skip the header row

    for row in csv_reader:
        # Extract station name and year data
        station_name = row[0]
        years_data = row[3:8]  # Extract years 2016 to 2020 (columns 4 to 8)

        # Remove commas and convert to integers
        years_data = [int(year.replace(',', '')) for year in years_data]

        # Calculate the sum for years 2016 to 2020
        sum_2016_to_2020 = sum(years_data)

        # Store the sum in the dictionary using station name as the key
        station_rider_sums[station_name] = sum_2016_to_2020
# print(station_rider_sums)

In [10]:
def min_max_normalize(input_dict):
    min_value = min(input_dict.values())
    max_value = max(input_dict.values())
    normalized_dict = {key: (value - min_value) / (max_value - min_value) for key, value in input_dict.items()}
    return normalized_dict

Cleaning Data

In [12]:
# cleaning it
modified_coordinates = {}
modified_station_rider_sums = {}

# Initialize a list to store missing keys
missing_keys = []

# Iterate through the keys in coordinates
for key in coordinates.keys():
    matching_substring = None

    # Check if the key exists as a substring in the keys of station_rider_sums
    for s in station_rider_sums.keys():
        if key in s:
            matching_substring = key
            modified_station_rider_sums[matching_substring] = station_rider_sums[s]
            break  # Exit the loop if a matching substring is found

    if matching_substring:
        modified_coordinates[matching_substring] = coordinates[key]
    else:
        missing_keys.append(key)

coordinates = modified_coordinates
station_rider_sums = modified_station_rider_sums

#normalize
station_rider_sums = min_max_normalize(station_rider_sums)
# print(station_rider_sums)


**API to calculate distance between nodes (stations)**


In [ ]:
# import requests
# import csv

# # Create and write to the CSV file
# with open("/content/output.csv", mode="w", newline="") as file:
#       writer = csv.writer(file)
#       writer.writerow(["start", "end", "duration (mins)"])


# with open("/content/output.csv", mode="a", newline="") as file:
#     writer = csv.writer(file)

#     for start in coordinates.items():
#       start_lat = start[1][0]
#       start_lon = start[1][1]
#       start_name = start[0]
#       print(start_name)
#       for end in coordinates.items():

#         end_name = end[0]
#         if end_name == start_name:
#           continue

#         end_lat = end[1][0]
#         end_lon = end[1][1]

#         url = f"https://api.mapbox.com/directions/v5/mapbox/cycling/{start_lon}%2C{start_lat}%3B{end_lon}%2C{end_lat}?alternatives=false&annotations=distance%2Cduration&continue_straight=true&geometries=geojson&overview=full&steps=false&access_token=pk.eyJ1Ijoic2FzaGEtc3NtOTk2MCIsImEiOiJjbG4wcm9saG8xc3duMmlxd3pjY2MybDcyIn0.ol8PlZNYBX_XbtBu8I3igw"

#         try:
#             r = requests.get(url)
#             r.raise_for_status()  # Raise an exception for bad response status
#             dur = r.json()['routes'][0]['duration'] / 60  # Duration in minutes

#         # Write the record to the CSV file immediately after receiving the response
#             writer.writerow([start_name, end_name, dur])
#         except requests.exceptions.RequestException as e:
#             print(f"Error making API request: {e}")


# print("CSV file 'output.csv' has been created and populated with data.")


Travel time between each station by cycling

In [14]:
stations_dic = {}
input_file_path = "data/stations.csv"

with open(input_file_path, "r") as input_file:
    next(input_file, None)
    for line in input_file:
        fields = line.strip().split(",")
        start = fields[0]
        end = fields[1]

        if start in coordinates  and end in coordinates:
          duration = float(fields[2])
          key = (start, end)
          stations_dic[key] = duration
# print(stations_dic)

Distance to the nearest station (g)

In [15]:
min_station_dist = {}

for station1 in stations_dic.items():
  start = station1[0][0]
  min_dist = 1000

  # getting the distance to all the stations from 'start'
  for station2 in stations_dic.items():

    cur_start = station2[0][0]

    if start != cur_start:
      continue

    min_dist = min(min_dist, station2[1])

  # saving the minimum in teh dictionary
  min_station_dist[start] = min_dist

# print(min_station_dist)


Dictionaries

In [18]:
# normalizing f and g dictionaries
# stations_dic = min_max_normalize(stations_dic) # f
min_station_dist = min_max_normalize(min_station_dist) # g


# creating and cleaning a node dictionary
node_dictionary = {}
for item in stations_dic.items():
  node_dictionary[item[0][0]] = 1
  node_dictionary[item[0][1]] = 1

node_dic = {}

for item in node_dictionary.items():
  node_dic[item[0]] = { 'name':item[0] ,
                        'f': station_rider_sums[item[0]] ,
                        'g': min_station_dist[item[0]] }

C = 0.01
D = 0.01

temp = {node: {"name": attrs["name"], "c": C*attrs["f"] + D*attrs["g"]}
                 for node, attrs in node_dic.items()}
node_dic = temp

#node index dictionary
index_dic = {}

index = 0
for node in node_dic:
  index_dic[node] = index
  index+=1

#edge dictionary
edge_dic = {}

# edges only between nodes w 30 min distance
for item in stations_dic.items():
  dur = item[1]
  if dur <= 40:
    edge_dic[item[0]] = {'cost': dur}

Creating the graph

In [19]:
import rustworkx
import matplotlib.pyplot as plt
from rustworkx.visualization import mpl_draw

graph = rustworkx.PyGraph()

for node in node_dic.items():
  graph.add_node(node[1])

for edge in edge_dic.items():
  graph.add_edge( index_dic[edge[0][0]] , index_dic[edge[0][1]] , edge[1]["cost"])

# Calculate the betweenness centrality for each node
bw_centrality = rustworkx.betweenness_centrality(graph)

# Create a larger figure with a specified size (adjust the values as needed)
fig = plt.figure(figsize=(200, 200))

# Create a subplot within the larger figure
subax1 = plt.subplot(121)

# Now, you can draw your graph with_labels=True on the larger subplot
mpl_draw(graph, with_labels=True, ax=subax1)

# Show the plot
plt.show()

Visualizing Centraliy

In [12]:
import matplotlib.pyplot as plt

# Generate a color list
colors = []
for node in graph.node_indices():
    colors.append(bw_centrality[node])

# Create a larger figure with a specified size
plt.figure(figsize=(200, 200))

# Generate a visualization with a colorbar
plt.rcParams['figure.figsize'] = [15, 10]
ax = plt.gca()
sm = plt.cm.ScalarMappable(norm=plt.Normalize(
    vmin=min(bw_centrality.values()),
    vmax=max(bw_centrality.values())
))
plt.colorbar(sm, ax=ax)
plt.title("Betweenness Centrality : ")

# Draw the graph with enlarged size
mpl_draw(graph, node_color=colors, ax=ax)

# Show the plot
plt.show()


In [35]:
####################################################################
#Formulating and solving a Hamiltonian from a given rustworkx graph#
####################################################################

from pyqubo import Binary
from pyqubo import Array
from pprint import pprint
import neal

nodes = len(node_dic)

#create an array of binary variables in our Hamiltonian.
#x[i] = 1 if a sensor is placed at that node, 0 otherwise
x = Array.create('x', shape=(nodes), vartype='BINARY')

#####calculate H_1#####
H_1 = 0
weight_max = None

for edge in graph.edge_list():

  # here weight (cost) is betweenness centrality of nodes in the edge
  weight = bw_centrality[edge[0]] + bw_centrality[edge[1]]

  if weight_max is None or weight > weight_max:
    weight_max = weight

  H_1 += (1-x[edge[0]])*(1-x[edge[1]])*weight

H_1 *= 1/weight_max

#####calculate H_2#####
H_2 = 0

for i in range(len(index_dic)):
  # here cost c = Cf(i) + Dg(i)
  cost = graph[i]["c"]
  H_2 += x[i]*cost

#####calculate H_3#####

#number of sensors we want to place
docks = 2
H_3 = (sum(x)-docks)**2

#####Get Hamiltonian function#####
A,B,C = 100,100,100 #random coef for now
H = A*H_1 + B*H_2 + C*H_3


In [36]:
#Compile the Hamiltonian to get a model
model = H.compile()

#Solve BinaryQuadraticModel(BQM) by using Sampler class
bqm = model.to_bqm()

#get the solutions of QUBO as SampleSet
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10)

#interpret the sampleset object
decoded_samples = model.decode_sampleset(sampleset)
#print the decoded_samples
#print("Printing the decoded sample:")
#pprint(decoded_samples)

best_sample = min(decoded_samples, key=lambda x: x.energy)

#print the best sample
print("Printing the best sample:")
pprint(best_sample.sample)

Printing the best sample:
{'x[0]': 0,
 'x[10]': 0,
 'x[11]': 0,
 'x[12]': 0,
 'x[13]': 0,
 'x[14]': 0,
 'x[15]': 1,
 'x[16]': 1,
 'x[17]': 0,
 'x[18]': 0,
 'x[19]': 0,
 'x[1]': 0,
 'x[20]': 0,
 'x[21]': 0,
 'x[22]': 0,
 'x[23]': 0,
 'x[24]': 1,
 'x[25]': 0,
 'x[26]': 0,
 'x[27]': 0,
 'x[28]': 0,
 'x[29]': 0,
 'x[2]': 0,
 'x[30]': 0,
 'x[31]': 0,
 'x[32]': 1,
 'x[33]': 0,
 'x[34]': 0,
 'x[35]': 0,
 'x[36]': 0,
 'x[37]': 0,
 'x[38]': 0,
 'x[39]': 0,
 'x[3]': 0,
 'x[40]': 0,
 'x[41]': 0,
 'x[42]': 0,
 'x[43]': 0,
 'x[44]': 0,
 'x[45]': 0,
 'x[46]': 0,
 'x[47]': 0,
 'x[48]': 0,
 'x[49]': 0,
 'x[4]': 0,
 'x[50]': 0,
 'x[51]': 1,
 'x[52]': 1,
 'x[53]': 0,
 'x[54]': 0,
 'x[55]': 0,
 'x[56]': 0,
 'x[57]': 0,
 'x[58]': 0,
 'x[59]': 0,
 'x[5]': 0,
 'x[60]': 0,
 'x[61]': 0,
 'x[62]': 0,
 'x[63]': 0,
 'x[64]': 0,
 'x[65]': 0,
 'x[66]': 0,
 'x[67]': 0,
 'x[68]': 0,
 'x[69]': 0,
 'x[6]': 0,
 'x[70]': 0,
 'x[71]': 0,
 'x[72]': 0,
 'x[73]': 0,
 'x[74]': 1,
 'x[75]': 0,
 'x[76]': 0,
 'x[77]': 0,
 'x[78

In [37]:
from qiskit.utils import algorithm_globals
from qiskit.algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np

from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model

<ipython-input-37-b9f1a98557f3>:2: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver


In [38]:
#Compile the Hamiltonian to get a model
model = H.compile()

#qubo dictionary from pyqubo
qubo_dict = model.to_qubo()

In [39]:
def create_problem(qubo_dict , nodes) -> QuadraticProgram:
    result = []
    mdl = Model()

    x = [mdl.binary_var("x%s" % i) for i in range(nodes)]

    res = []
    for (var1, var2), coef in qubo_dict.items():

      #gettinggthe index from the var strings
      v1 = int(var1[2:len(var1)-1])
      v2 = int(var2[2:len(var2)-1])

      res.append(x[v1] * x[v2] * coef)

    objective = mdl.sum(res)
    mdl.minimize(objective)
    cost = (mdl.sum(x))
    #mdl.add_constraint(cost == total)

    qp = from_docplex_mp(mdl)
    return qp

qubo = create_problem(qubo_dict[0], len(node_dic))
print(qubo.prettyprint())


Problem name: docplex_model1

Minimize
  -851.7506106224324*x0^2 + 202.57222256591575*x0*x1 + 200*x0*x10 + 200*x0*x11
  + 200*x0*x12 + 200*x0*x13 + 200*x0*x14 + 305.27157918129797*x0*x15
  + 200*x0*x16 + 202.29560404677*x0*x17 + 200*x0*x18 + 212.09685137781108*x0*x19
  + 210.33427632983057*x0*x2 + 202.54063031876507*x0*x20 + 200*x0*x21
  + 200*x0*x22 + 200*x0*x23 + 200*x0*x24 + 211.3207927100799*x0*x25
  + 202.25996032504037*x0*x26 + 208.38252117855012*x0*x27 + 200*x0*x28
  + 200*x0*x29 + 200*x0*x3 + 220.96033005103052*x0*x30 + 200*x0*x31
  + 282.34984652599303*x0*x32 + 207.38223160682134*x0*x33 + 200*x0*x34
  + 200*x0*x35 + 200*x0*x36 + 200*x0*x37 + 200*x0*x38 + 200*x0*x39 + 200*x0*x4
  + 200*x0*x40 + 200*x0*x41 + 200*x0*x42 + 200*x0*x43 + 200*x0*x44 + 200*x0*x45
  + 200*x0*x46 + 200*x0*x47 + 200*x0*x48 + 200*x0*x49 + 200*x0*x5 + 200*x0*x50
  + 247.5032964054443*x0*x51 + 298.2771938741511*x0*x52 + 200*x0*x53
  + 200*x0*x54 + 200*x0*x55 + 200*x0*x56 + 201.87558905999362*x0*x57
  + 208.

Translating this QUBO into an Ising operator

In [ ]:
op, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

In [ ]:
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0.0, 0.0])
exact_mes = NumPyMinimumEigensolver()

In [ ]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

MinimumEigenOptimizer based on the classical exact NumPyMinimumEigensolver.

In [ ]:
exact_result = exact.solve(qubo)
print(exact_result.prettyprint())

MinimumEigenOptimizer based on QAOA to the same problem.

In [ ]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result.prettyprint())